In [1]:
import pandas as pd
import pymongo
from IPython.display import clear_output
from geopy.geocoders import Nominatim
import json
import re
import string
import concurrent.futures
import requests

In [12]:
# Connect to database
connection_string='mongodb://localhost:27017'
client = pymongo.MongoClient(connection_string)
# Define database for storage
db = client.dice_db
# Retrieve data for analysis
jobs_collection = db.jobs.find()

In [13]:
#get mongoDB data into a dataframe
jobs_df = pd.DataFrame(list(jobs_collection))
jobs_df

,_id,job_title,job_company,job_salary,job_location,job_date,job_type,job_description
0,5fd484fef896a84fc0f76c25,Business Analytics Director,University of Kentucky,Depends on Experience,"Lexington, KY, USA",2020-12-12,Full Time,\nThe University of Kentucky’s Institutional R...
1,5fd48500f896a84fc0f76c26,Associate Software Engineer III,Penske,Depends on Experience,"Reading, PA, USA",2020-12-12,Full Time,\nSummary Statement:Working with a diverse tea...
2,5fd48502f896a84fc0f76c27,Developer,Healh Care Logistics,"$70,000 - $85,000","Grove City, OH, USA",2020-12-11,Full Time,\nTitle: Developer ...
3,5fd48504f896a84fc0f76c28,ServiceNow Solution Architect,Saxon Global Inc.,$open,"Charlotte, NC, USA",2020-12-13,Full Time,"\nHi, Please go through the job description an..."
4,5fd48506f896a84fc0f76c29,Full Time HRIS Developer,Saxon Global Inc.,Depends On Experience,"Nashville, TN, USA",2020-12-13,Full Time,"\nLocation: Nashville TN, Memphis TN , Ridgela..."
...,...,...,...,...,...,...,...,...
17841,5fd6636d9816b2c8cd252ee4,Software Engineer,Motion Recruitment,,"Dallas, TX, USA",2020-12-13,Full Time,\n\t\t\tOne of the world's largest retailers i...
17842,5fd6636f9816b2c8cd252ee5,Java/Golang Software Developer,Motion Recruitment,,"Irvine, CA, USA",2020-12-13,Full Time,\nJava/Golang Software Developer Job Descripti...
17843,5fd663719816b2c8cd252ee6,Software Architect / $200k / Java Microservices,Motion Recruitment,,"Los Angeles, CA, USA",2020-12-13,Full Time,\n\t\t\tOne of the world's oldest recording la...
17844,5fd663739816b2c8cd252ee7,UX / Product Designer,Motion Recruitment,,"San Francisco, CA, USA",2020-12-13,Full Time,\n\t\t\tA consulting agency in San Francisco i...


In [14]:
#get a summary of the data
jobs_df.describe()

,_id,job_title,job_company,job_salary,job_location,job_date,job_type,job_description
count,17846,17846,17846,17846,17846,17846,17846,17846
unique,17846,9394,459,803,1922,46,53,12755
top,5fd5b436baa823c8d8b6997b,Software Engineer,Apex Systems,,"New York, NY, USA",2020-12-13,Full Time,\nCX Researcher / Journey Mapper for Global Se...
freq,1,138,2132,12582,664,9111,16605,29


In [11]:
#drop duplicate rows
jobs_df = jobs_df.drop("_id", axis=1).drop_duplicates("job_description")
jobs_df.describe()

,job_title,job_company,job_salary,job_location,job_date,job_type,job_description
count,12304,12304,12304,12304,12304,12304,12304
unique,8965,448,798,1534,23,53,12304
top,Software Engineer,CyberCoders,,"New York, NY, USA",2020-12-13,Full Time,\nDescription This is Opportunity to grow your...
freq,111,1541,8345,454,7228,11556,1


In [ ]:
jobs_df=jobs_df[~jobs_df.job_description.isin(old_jobs_df.job_description)]

In [7]:
# read the previously recorded jobs
old_jobs_df = pd.read_csv("ProcessedData.csv")\
    [["job_title", "job_company", "job_salary", "job_location", "job_date", "job_description"]]

# keep only the jobs that are new
# jobs_df = jobs_df.append(old_jobs_df).drop_duplicates("job_description", keep=False)\
#     .reset_index().drop("index", axis=1).dropna() #correct this (it only drops all duplicates but doesn't keep only the new values)
jobs_df=jobs_df[~jobs_df.job_description.isin(old_jobs_df.job_description)]
jobs_df

,job_title,job_company,job_salary,job_location,job_date,job_type,job_description
0,Business Analytics Director,University of Kentucky,Depends on Experience,"Lexington, KY, USA",2020-12-12,Full Time,\nThe University of Kentucky’s Institutional R...
1,Associate Software Engineer III,Penske,Depends on Experience,"Reading, PA, USA",2020-12-12,Full Time,\nSummary Statement:Working with a diverse tea...
2,Developer,Healh Care Logistics,"$70,000 - $85,000","Grove City, OH, USA",2020-12-11,Full Time,\nTitle: Developer ...
3,ServiceNow Solution Architect,Saxon Global Inc.,$open,"Charlotte, NC, USA",2020-12-13,Full Time,"\nHi, Please go through the job description an..."
4,Full Time HRIS Developer,Saxon Global Inc.,Depends On Experience,"Nashville, TN, USA",2020-12-13,Full Time,"\nLocation: Nashville TN, Memphis TN , Ridgela..."
...,...,...,...,...,...,...,...
16807,C++ Developer (GAMING),CyberCoders,-,"Irvine, CA, USA",2020-12-13,Full Time,"\n\t\t\tMinimum Required Skills:C++, Gaming, U..."
16808,Senior Electrical Engineer - FPGA,CyberCoders,-,"Rockville, MD, USA",2020-12-13,Full Time,"\n\t\t\tMinimum Required Skills:FPGA, Hardware..."
16809,Sr Process Development Engineer,CyberCoders,-,"Oceanport, NJ, USA",2020-12-13,Full Time,\n\t\t\tMinimum Required Skills:Printed Electr...
16811,Developer,CyberCoders,"$80,000 - $120,000","Indianapolis, IN, USA",2020-12-13,Full Time,"\n\t\t\tMinimum Required Skills:Artiva, OOD, O..."


In [7]:
# Apply a first round of text cleaning techniques
re.DOTALL
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)

    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', ' ', text)
    text = re.sub('\n', ' ', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [8]:
# Apply the data cleaning
jobs_clean_df = jobs_df
jobs_clean_df = pd.DataFrame(jobs_clean_df["job_description"].apply(round1))

In [9]:
# See description before transformation
jobs_df.loc[1, "job_description"]

"CBO Data Control Application Analyst perform various interviewing, fact collection and financial evaluation tasks in connection with credit and collection in a large and diversified hospital serving both non-indigent and indigent patients qualifying for various types of specialized assistance programs. Incumbents may specialize in either credit or collection work or a combination of both. Primary emphasis is on financial counseling with patient from pre-admission through account becoming a discharged zero balance. Incumbents are responsible for interview of patient, family or guarantor to explain charges, hospital policy and payment procedures. Patient Finance Associates are responsible for confirming all commercial insurance and conducting extensive third party and welfare program finding in order to obtain payments as promptly as possible or direct account to collection agency.\n\nQualifications\n\nExperience: Generally requires 3 to 5 years of related experience.\n\nEducation: High

In [10]:
# See description after transformation
jobs_clean_df.loc[1, "job_description"]

'cbo data control application analyst perform various interviewing  fact collection and financial evaluation tasks in connection with credit and collection in a large and diversified hospital serving both non indigent and indigent patients qualifying for various types of specialized assistance programs  incumbents may specialize in either credit or collection work or a combination of both  primary emphasis is on financial counseling with patient from pre admission through account becoming a discharged zero balance  incumbents are responsible for interview of patient  family or guarantor to explain charges  hospital policy and payment procedures  patient finance associates are responsible for confirming all commercial insurance and conducting extensive third party and welfare program finding in order to obtain payments as promptly as possible or direct account to collection agency   qualifications  experience  generally requires   to   years of related experience   education  high schoo

In [11]:
key_words = ["data+analyst", "data+scientist", "excel", "python", "pandas", "matplotlib", "sql", "postgresql", "bootstrap", "nosql", \
    "mongodb", "mongo", "javascript", "tableau", "machine+learning", "ml", "scikit+learn", "scikit", "keras", "tensorflow", "pyspark", "natural+language+processing", \
        "nlp", "big+data", "etl", "extract+transform+load", "amazon+web+services", "aws", "rds"]

In [12]:
#clean-up characters in keywords
key_words = map(round1, key_words)
key_words = [key for key in key_words]
key_words

['data analyst',
 'data scientist',
 'excel',
 'python',
 'pandas',
 'matplotlib',
 'sql',
 'postgresql',
 'bootstrap',
 'nosql',
 'mongodb',
 'mongo',
 'javascript',
 'tableau',
 'machine learning',
 'ml',
 'scikit learn',
 'scikit',
 'keras',
 'tensorflow',
 'pyspark',
 'natural language processing',
 'nlp',
 'big data',
 'etl',
 'extract transform load',
 'amazon web services',
 'aws',
 'rds']

In [13]:
# We are going to create a document-term matrix using CountVectorizer, and exclude common English stop words
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(vocabulary=key_words, ngram_range=(1, 3))
data_cv = cv.fit_transform(jobs_clean_df["job_description"])
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = jobs_clean_df.index
data_dtm

,data analyst,data scientist,excel,python,pandas,matplotlib,sql,postgresql,bootstrap,nosql,...,tensorflow,pyspark,natural language processing,nlp,big data,etl,extract transform load,amazon web services,aws,rds
0,2,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
481,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
482,1,0,2,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
483,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# count words for all jobs to find key words
total_words = pd.DataFrame()
for c in data_dtm.columns:
    total_words.loc[0, c] = data_dtm[c].sum()
    
# Transpose and sort
word_totals = total_words.transpose().reset_index().rename(columns={"index": "word", 0:"count"}).sort_values("count",ascending=False)
word_totals

,word,count
6,sql,311.0
2,excel,233.0
0,data analyst,207.0
13,tableau,133.0
3,python,89.0
24,etl,36.0
14,machine learning,30.0
23,big data,24.0
27,aws,19.0
12,javascript,11.0


In [15]:
# Join all data into base dataframe
jobs_df = jobs_df.join(data_dtm).join(pd.DataFrame(data_dtm.sum(axis=1))).rename(columns={0:"Sum_of_Counts"})
jobs_df

,job_title,job_company,job_salary,job_location,job_date,job_description,data analyst,data scientist,excel,python,...,pyspark,natural language processing,nlp,big data,etl,extract transform load,amazon web services,aws,rds,Sum_of_Counts
0,Data Analyst,FIRST CHOICE COMMUNITY HEALTHCARE,,"Albuquerque, NM 87105",2020-11-07,Under the supervision of the Chief Operations ...,2,0,0,0,...,0,0,0,0,0,0,0,0,0,3
1,Data Ctrl App Analyst CBO,JACKSON HEALTH SYSTEM,,"Miami, FL",2020-11-07,CBO Data Control Application Analyst perform v...,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Quality Data Analyst,VISITING NURSE SERVICE OF NEW YORK,,"Manhattan, NY",2020-11-07,Overview\n\nProvides data and informatics supp...,0,0,1,0,...,0,0,0,0,0,0,0,0,0,2
5,Data Analyst - HSS ACO Grant,Providence,,"Anaheim, CA 92805",2020-11-07,We are looking for a Data Analyst with the HSS...,2,0,1,0,...,0,0,0,0,0,0,0,0,0,4
6,eCommerce Sales Analyst,Emerson,,"Elyria, OH",2020-11-07,"At Emerson, we are innovators and problem-solv...",0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,Data Analyst/Business Analyst,HGcomply,$35.00 - $38.50 an hour,"Pittsburgh, PA 15209",2020-11-06,BUSINESS DATA ANALYST W2 PROJECT!We are active...,2,0,0,0,...,0,0,0,0,0,0,0,0,0,2
481,Data Analyst,EARLY LEARNING COALITION OF HILLSBOROUGH COUNTY,,"Tampa, FL 33619",2020-11-06,This position is responsible for collecting an...,1,0,0,0,...,0,0,0,0,0,0,0,0,0,3
482,Marketing Data Analyst,SHAR MUSIC,$15 - $42 an hour,"Ann Arbor, MI 48104",2020-11-06,Our Marketing Data Analyst's primary function ...,1,0,2,0,...,0,0,0,0,0,0,0,0,0,5
483,Data and Assessment Analyst (Education),Beginning with Children Charter School 2,"$53,000 - $63,000 a year","Brooklyn, NY",2020-11-06,Beginning with ChildrenBeginning with Children...,1,0,1,0,...,0,0,0,0,0,0,0,0,0,3


In [16]:
# Define function to find coordinates
def find_coordinates(address): 
    address = address+", USA"
    clear_output(wait=True)
    find_coordinates.counter+=1 
    print(f'counter: {find_coordinates.counter} / {find_coordinates.total_iterations}')
    # Try with address as is
    try:
        geolocator = Nominatim(user_agent="DreamTeam")
        location = geolocator.geocode(address)
        print(f'address: {address}')
        print(f'LatLng:{location.latitude, location.longitude}')

        return location.latitude, location.longitude
    except:
        # Remove text from parantesis
        try:
            altered_address_no_parantesis = re.sub(r'\([^)]*\)', '', address)
            geolocator = Nominatim(user_agent="DreamTeam")
            location = geolocator.geocode(altered_address_no_parantesis)
            print(f'address: {altered_address_no_parantesis}')
            print(f'LatLng:{location.latitude, location.longitude}')
            return location.latitude, location.longitude            
        except:
            try:
                address_only_number = re.findall('\d+', address)[0]
                geolocator = Nominatim(user_agent="DreamTeam")
                location = geolocator.geocode(address_only_number)
                #if longitude limits are ourside US, do not consider the number, which is supposed to be the ZIP
                if location.longitude>-60:
                    address_wo_number = re.sub("^\d+\s|\s\d+,\s|\s\d+$", ", ", address)
                    geolocator = Nominatim(user_agent="DreamTeam")
                    location = geolocator.geocode(address_wo_number)     
                    print(f'address: {address_wo_number}')
                    print(f'LatLng:{location.latitude, location.longitude}')
                else:
                    print(f'address: {address_only_number}')
                    print(f'LatLng:{location.latitude, location.longitude}')
                return location.latitude, location.longitude
            except:
                return "N/A", "N/A"

In [17]:
# # Test address
# find_coordinates.counter = 0
# find_coordinates.total_iterations = 1
# #find_coordinates("Mountain View, CA 94042")
# find_coordinates("Mountain View, CA 94042")

In [18]:
# Get coordinates for locations
find_coordinates.counter = 0
location_list = jobs_df["job_location"].to_list()
find_coordinates.total_iterations = len(location_list)
# Run multithreaded request
with concurrent.futures.ThreadPoolExecutor(max_workers=1) as executor:
    coord_list = executor.map(find_coordinates, location_list)
    coord_list = [x for x in coord_list]

counter: 431 / 431
address: Chesterfield, MO, USA
LatLng:(38.661953, -90.5643258)


In [19]:
# Record coordinates into clean dataframe
coord_list_transposed = zip(*coord_list)
coord_list_transposed = [i for i in coord_list_transposed]
jobs_df["latitudes"] = coord_list_transposed[0]
jobs_df["longitudes"] = coord_list_transposed[1]
jobs_df

,job_title,job_company,job_salary,job_location,job_date,job_description,data analyst,data scientist,excel,python,...,nlp,big data,etl,extract transform load,amazon web services,aws,rds,Sum_of_Counts,latitudes,longitudes
0,Data Analyst,FIRST CHOICE COMMUNITY HEALTHCARE,,"Albuquerque, NM 87105",2020-11-07,Under the supervision of the Chief Operations ...,2,0,0,0,...,0,0,0,0,0,0,0,3,35.084103,-106.650985
1,Data Ctrl App Analyst CBO,JACKSON HEALTH SYSTEM,,"Miami, FL",2020-11-07,CBO Data Control Application Analyst perform v...,0,0,0,0,...,0,0,0,0,0,0,0,0,25.774266,-80.193659
2,Quality Data Analyst,VISITING NURSE SERVICE OF NEW YORK,,"Manhattan, NY",2020-11-07,Overview\n\nProvides data and informatics supp...,0,0,1,0,...,0,0,0,0,0,0,0,2,40.789624,-73.959894
5,Data Analyst - HSS ACO Grant,Providence,,"Anaheim, CA 92805",2020-11-07,We are looking for a Data Analyst with the HSS...,2,0,1,0,...,0,0,0,0,0,0,0,4,33.834752,-117.911732
6,eCommerce Sales Analyst,Emerson,,"Elyria, OH",2020-11-07,"At Emerson, we are innovators and problem-solv...",0,0,0,0,...,0,0,0,0,0,0,0,0,41.368251,-82.106576
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,Data Analyst/Business Analyst,HGcomply,$35.00 - $38.50 an hour,"Pittsburgh, PA 15209",2020-11-06,BUSINESS DATA ANALYST W2 PROJECT!We are active...,2,0,0,0,...,0,0,0,0,0,0,0,2,40.441694,-79.990086
481,Data Analyst,EARLY LEARNING COALITION OF HILLSBOROUGH COUNTY,,"Tampa, FL 33619",2020-11-06,This position is responsible for collecting an...,1,0,0,0,...,0,0,0,0,0,0,0,3,27.953105,-82.450704
482,Marketing Data Analyst,SHAR MUSIC,$15 - $42 an hour,"Ann Arbor, MI 48104",2020-11-06,Our Marketing Data Analyst's primary function ...,1,0,2,0,...,0,0,0,0,0,0,0,5,42.268157,-83.731229
483,Data and Assessment Analyst (Education),Beginning with Children Charter School 2,"$53,000 - $63,000 a year","Brooklyn, NY",2020-11-06,Beginning with ChildrenBeginning with Children...,1,0,1,0,...,0,0,0,0,0,0,0,3,40.650104,-73.949582


In [20]:
# jobs_df = pd.read_csv("ProcessedData.csv")
# jobs_df = jobs_df.drop("Unnamed: 0", axis=1)

In [21]:
# Function to split the salary text
def split_salary(string):
    
    try:
        amount = re.findall('(\$\d+,\d+|\$\d+)', string)
        salary_min = amount[0]
        salary_min = re.findall("\d+,\d+|\d+",salary_min)[0]
        try:
            salary_max = amount[1]#.group(0)
            salary_max = re.findall("\d+,\d+|\d+",salary_max)[0]
        except:
            salary_max = ""
        um = re.findall('hour|year', string)  
        um = "$/"+um[0]
        return (salary_min, salary_max, um)
    except:
        return ("","","")

In [22]:
# Split salary text into minimum salary, maximum salary and unit of measurement
salary_list = jobs_df["job_salary"].astype(str).values.tolist()
salary_list = map(split_salary, salary_list)
salary_list = [x for x in salary_list]

In [23]:
# Record splitted salaries into clean dataframe
salary_list_transposed = zip(*salary_list)
salary_list_transposed = [i for i in salary_list_transposed]
jobs_df["salary_min"] = salary_list_transposed[0]
jobs_df["salary_max"] = salary_list_transposed[1]
jobs_df["UM"] = salary_list_transposed[2]
jobs_df

,job_title,job_company,job_salary,job_location,job_date,job_description,data analyst,data scientist,excel,python,...,extract transform load,amazon web services,aws,rds,Sum_of_Counts,latitudes,longitudes,salary_min,salary_max,UM
0,Data Analyst,FIRST CHOICE COMMUNITY HEALTHCARE,,"Albuquerque, NM 87105",2020-11-07,Under the supervision of the Chief Operations ...,2,0,0,0,...,0,0,0,0,3,35.084103,-106.650985,,,
1,Data Ctrl App Analyst CBO,JACKSON HEALTH SYSTEM,,"Miami, FL",2020-11-07,CBO Data Control Application Analyst perform v...,0,0,0,0,...,0,0,0,0,0,25.774266,-80.193659,,,
2,Quality Data Analyst,VISITING NURSE SERVICE OF NEW YORK,,"Manhattan, NY",2020-11-07,Overview\n\nProvides data and informatics supp...,0,0,1,0,...,0,0,0,0,2,40.789624,-73.959894,,,
5,Data Analyst - HSS ACO Grant,Providence,,"Anaheim, CA 92805",2020-11-07,We are looking for a Data Analyst with the HSS...,2,0,1,0,...,0,0,0,0,4,33.834752,-117.911732,,,
6,eCommerce Sales Analyst,Emerson,,"Elyria, OH",2020-11-07,"At Emerson, we are innovators and problem-solv...",0,0,0,0,...,0,0,0,0,0,41.368251,-82.106576,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,Data Analyst/Business Analyst,HGcomply,$35.00 - $38.50 an hour,"Pittsburgh, PA 15209",2020-11-06,BUSINESS DATA ANALYST W2 PROJECT!We are active...,2,0,0,0,...,0,0,0,0,2,40.441694,-79.990086,35,38,$/hour
481,Data Analyst,EARLY LEARNING COALITION OF HILLSBOROUGH COUNTY,,"Tampa, FL 33619",2020-11-06,This position is responsible for collecting an...,1,0,0,0,...,0,0,0,0,3,27.953105,-82.450704,,,
482,Marketing Data Analyst,SHAR MUSIC,$15 - $42 an hour,"Ann Arbor, MI 48104",2020-11-06,Our Marketing Data Analyst's primary function ...,1,0,2,0,...,0,0,0,0,5,42.268157,-83.731229,15,42,$/hour
483,Data and Assessment Analyst (Education),Beginning with Children Charter School 2,"$53,000 - $63,000 a year","Brooklyn, NY",2020-11-06,Beginning with ChildrenBeginning with Children...,1,0,1,0,...,0,0,0,0,3,40.650104,-73.949582,"53,000","63,000",$/year


In [24]:
# Store data to CSV (append new data to old data)
jobs_df.to_csv("ProcessedData.csv", mode='a', header=False)

In [25]:
# # Insert clean data into new MongoDB table
# #db.jobs_clean.drop()
# clean_records = json.loads(jobs_df.T.to_json()).values()
# db.jobs_clean.insert_many(clean_records)